In [5]:
!pip install torch torchvision tqdm
!pip install --upgrade ipywidgets
!pip install --upgrade tqdm


You should consider upgrading via the 'C:\Users\zelkh\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\zelkh\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\zelkh\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [8]:
# MODEL (with training recipe upgrades)

# MODEL + TRAIN (CIFAR-10, poly-softplus, SGD+cosine, MixUp, EMA)

import math, copy, numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from tqdm.notebook import tqdm
from torch.optim.lr_scheduler import CosineAnnealingLR, SequentialLR, LinearLR

# ----------------------------------
# 1) DATA
# ----------------------------------
transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                         (0.2023, 0.1994, 0.2010))
])

batch_size   = 128
num_workers  = 2
use_cuda     = torch.cuda.is_available()
device       = torch.device('cuda' if use_cuda else 'cpu')
pin_mem_flag = True if use_cuda else False

trainset   = torchvision.datasets.CIFAR10(root='./data', train=True,  download=True, transform=transform)
testset    = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True,
                                          num_workers=num_workers, pin_memory=pin_mem_flag)
testloader  = torch.utils.data.DataLoader(testset,  batch_size=batch_size, shuffle=False,
                                          num_workers=num_workers, pin_memory=pin_mem_flag)

# ----------------------------------
# 2) POLY ACT (Softplus-like, degree 4) with learnable affine
# ----------------------------------
class PolyAct4(nn.Module):
    """
    y = s * (A x^4 + B x^3 + C x^2 + D x + E) + b, where x is clamped to [clamp_min, clamp_max].
    A..E are fixed buffers (from your fit); s,b are tiny learnable scalars to absorb residual scale/bias.
    """
    def __init__(self, A, B, C, D, E, clamp_min=-6.0, clamp_max=6.0, init_scale=1.0, init_bias=0.0):
        super().__init__()
        self.register_buffer('A', torch.tensor(float(A)))
        self.register_buffer('B', torch.tensor(float(B)))
        self.register_buffer('C', torch.tensor(float(C)))
        self.register_buffer('D', torch.tensor(float(D)))
        self.register_buffer('E', torch.tensor(float(E)))
        self.clamp_min = clamp_min
        self.clamp_max = clamp_max
        self.scale = nn.Parameter(torch.tensor(float(init_scale)))
        self.bias  = nn.Parameter(torch.tensor(float(init_bias)))

    def forward(self, x):
        x = torch.clamp(x, min=self.clamp_min, max=self.clamp_max)
        y = self.A * x**4 + self.B * x**3 + self.C * x**2 + self.D * x + self.E
        return self.scale * y + self.bias

# ----------------------------------
# 3) MODEL (VGG-ish)
# ----------------------------------
def conv3x3(in_ch, out_ch):
    # BN follows -> bias not needed
    return nn.Conv2d(in_ch, out_ch, kernel_size=3, stride=1, padding=1, bias=False)

class CIFAR_CNN(nn.Module):
    def __init__(self, activation_fn: nn.Module):
        super().__init__()
        act = activation_fn  # shared instance (keeps HE-compat simplicity)

        self.block1 = nn.Sequential(
            conv3x3(3, 48),  nn.BatchNorm2d(48),  act,
            conv3x3(48, 48), nn.BatchNorm2d(48),  act,
            nn.AvgPool2d(2, 2)   # 32 -> 16
        )

        self.block2 = nn.Sequential(
            conv3x3(48, 96),  nn.BatchNorm2d(96),  act,
            conv3x3(96, 96),  nn.BatchNorm2d(96),  act,
            nn.AvgPool2d(2, 2)   # 16 -> 8
        )

        self.block3 = nn.Sequential(
            conv3x3(96, 192),  nn.BatchNorm2d(192), act,
            conv3x3(192, 192), nn.BatchNorm2d(192), act
        )

        # optional light downsample to stabilize scales
        self.ds3 = nn.AvgPool2d(2, 2)  # 8 -> 4

        self.block4 = nn.Sequential(
            conv3x3(192, 256), nn.BatchNorm2d(256), act,
            conv3x3(256, 256), nn.BatchNorm2d(256), act
        )

        self.gap     = nn.AdaptiveAvgPool2d((1, 1))
        self.fc1     = nn.Linear(256, 512, bias=False)  # BN follows
        self.fc_bn   = nn.BatchNorm1d(512)
        self.head_act= activation_fn
        self.dropout = nn.Dropout(0.3)
        self.fc2     = nn.Linear(512, 10)

        # init
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.Linear):
                nn.init.kaiming_uniform_(m.weight, a=math.sqrt(5))

    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.ds3(x)
        x = self.block4(x)
        x = self.gap(x).flatten(1)
        x = self.fc1(x)
        x = self.fc_bn(x)
        x = self.head_act(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# ----------------------------------
# 4) Build model, loss, optimizer, sched
# ----------------------------------
poly4 = PolyAct4(
    A=-2.22734060e-03,
    B=3.70901207e-17,
    C=1.15754806e-01,
    D=5.00000000e-01,   # <-- corrected (0.5)
    E=7.00459235e-01,
    clamp_min=-6.0, clamp_max=6.0,
    init_scale=1.0, init_bias=0.0
)

model = CIFAR_CNN(activation_fn=poly4).to(device)

criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

base_lr        = 0.1
optimizer      = optim.SGD(model.parameters(), lr=base_lr, momentum=0.9,
                           weight_decay=5e-4, nesterov=True)
num_epochs     = 350
warmup_epochs  = 5
warmup_sched   = LinearLR(optimizer, start_factor=1e-3, end_factor=1.0, total_iters=warmup_epochs)
cosine_sched   = CosineAnnealingLR(optimizer, T_max=num_epochs - warmup_epochs, eta_min=1e-4)
scheduler      = SequentialLR(optimizer, schedulers=[warmup_sched, cosine_sched],
                              milestones=[warmup_epochs])

scaler = torch.cuda.amp.GradScaler(enabled=use_cuda)

# ----------------------------------
# 5) MixUp utilities (set alpha=0.0 to disable)
# ----------------------------------
MIXUP_ALPHA = 0.2

def mixup_data(x, y, alpha=MIXUP_ALPHA):
    if alpha <= 0.0:
        return x, y, 1.0, None
    lam = np.random.beta(alpha, alpha)
    bs  = x.size(0)
    idx = torch.randperm(bs, device=x.device)
    mixed_x = lam * x + (1 - lam) * x[idx]
    y_a, y_b = y, y[idx]
    return mixed_x, (y_a, y_b), lam, idx

def mixup_criterion(crit, pred, ytuple_lam):
    y_a, y_b, lam = ytuple_lam
    return lam * crit(pred, y_a) + (1 - lam) * crit(pred, y_b)

# ----------------------------------
# 6) EMA (deepcopy to avoid constructor args)
# ----------------------------------
class ModelEMA:
    def __init__(self, model, decay=0.999):
        self.ema = copy.deepcopy(model).to(device)
        for p in self.ema.parameters():
            p.requires_grad_(False)
        self.decay = decay

    @torch.no_grad()
    def update(self, model):
        d = self.decay
        msd = model.state_dict()
        for k, v in self.ema.state_dict().items():
            if v.dtype.is_floating_point:
                v.mul_(d).add_((1.0 - d) * msd[k])
            else:
                v.copy_(msd[k])

    def state_dict(self):
        return self.ema.state_dict()

ema = ModelEMA(model, decay=0.999)

# ----------------------------------
# 7) Eval helper (uses given model, e.g., ema.ema)
# ----------------------------------
@torch.no_grad()
def evaluate(eval_model):
    eval_model.eval()
    tot_loss, correct, total = 0.0, 0, 0
    for inputs, labels in testloader:
        inputs = inputs.to(device, non_blocking=pin_mem_flag)
        labels = labels.to(device, non_blocking=pin_mem_flag)
        with torch.cuda.amp.autocast(enabled=use_cuda):
            outputs = eval_model(inputs)
            loss = criterion(outputs, labels)
        tot_loss += loss.item()
        pred = outputs.argmax(dim=1)
        correct += (pred == labels).sum().item()
        total   += labels.size(0)
    return tot_loss / len(testloader), 100.0 * correct / total

# ----------------------------------
# 8) TRAIN
# ----------------------------------
for epoch in range(num_epochs):
    model.train()
    running = 0.0
    loop = tqdm(trainloader, desc=f'Epoch {epoch+1}/{num_epochs}', leave=False)
    for x, y in loop:
        x = x.to(device, non_blocking=pin_mem_flag)
        y = y.to(device, non_blocking=pin_mem_flag)

        x, (ya, yb), lam, _ = mixup_data(x, y, alpha=MIXUP_ALPHA)

        optimizer.zero_grad(set_to_none=True)
        with torch.cuda.amp.autocast(enabled=use_cuda):
            out  = model(x)
            loss = mixup_criterion(criterion, out, (ya, yb, lam))
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        ema.update(model)

        running += loss.item()
        loop.set_postfix(train_loss=loss.item(), lr=optimizer.param_groups[0]['lr'])

    scheduler.step()
    train_loss = running / len(trainloader)

    val_loss, val_acc = evaluate(ema.ema)   # eval EMA weights
    print(f"Epoch [{epoch+1}/{num_epochs}] | Train Loss: {train_loss:.4f} | "
          f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%")

# ----------------------------------
# 9) FINAL EVAL + SAVE (EMA)
# ----------------------------------
final_loss, final_acc = evaluate(ema.ema)
print(f"\nFinal Test Accuracy (EMA): {final_acc:.2f}%")

torch.save(ema.state_dict(), 'test_cifar10_trained_model_ema.pth')
print("Saved: test_cifar10_trained_model_ema.pth ✅")




Files already downloaded and verified
Files already downloaded and verified


C:\Users\zelkh\AppData\Local\Temp\ipykernel_50056\2198769688.py:155: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=use_cuda)


Epoch 1/350:   0%|          | 0/391 [00:00<?, ?it/s]

C:\Users\zelkh\AppData\Local\Temp\ipykernel_50056\2198769688.py:234: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=use_cuda):
C:\Users\zelkh\AppData\Local\Temp\ipykernel_50056\2198769688.py:211: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=use_cuda):


Epoch [1/350] | Train Loss: 2.1761 | Val Loss: 2.3675 | Val Acc: 10.00%


...

Epoch [61/350] | Train Loss: 1.2399 | Val Loss: 0.8929 | Val Acc: 84.31%


Epoch 62/350:   0%|          | 0/391 [00:00<?, ?it/s]

Epoch [62/350] | Train Loss: 1.2249 | Val Loss: 0.9100 | Val Acc: 84.45%


Epoch 63/350:   0%|          | 0/391 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [10]:
# CIFAR-10 | PreAct-ResNet-18 + Polynomial Softplus (deg-4)
# Option B: AdamW + cosine, aug schedule (light→strong→light), MixUp ramp→off, EMA, AMP, BN calibration

import os, math, copy, random
from dataclasses import dataclass

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.optim.lr_scheduler import CosineAnnealingLR
from tqdm.auto import tqdm

# =========================
# Config
# =========================
@dataclass
class CFG:
    epochs: int = 350
    batch_size: int = 128
    lr: float = 1e-3
    weight_decay: float = 3e-4
    momentum: float = 0.9
    amp: bool = torch.cuda.is_available()

    # polynomial activation clamp
    act_clip: float = 4.5  # tighter than 6.0 to reduce approx error

    # augmentation schedule
    phase1_epochs: int = 20      # LIGHT aug (no MixUp)
    phase3_ratio: float = 0.10   # last 10% epochs LIGHT aug again
    mixup_target_alpha: float = 0.2
    mixup_ramp_epochs: int = 5   # ramp 0 -> target at start of phase 2

    # EMA
    ema_decay: float = 0.999

    # saving
    save_std: str = "resnet_polyB_last.pth"
    save_ema: str = "resnet_polyB_ema.pth"

    seed: int = 42

cfg = CFG()

# =========================
# Repro & device
# =========================
def set_seed(s=42):
    random.seed(s); torch.manual_seed(s); torch.cuda.manual_seed_all(s)
set_seed(cfg.seed)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pin_mem = torch.cuda.is_available()
torch.backends.cudnn.benchmark = True

# =========================
# Data (light & strong pipelines)
# =========================
# LIGHT aug (start + end)
transform_train_light = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914,0.4822,0.4465),
                         (0.2023,0.1994,0.2010)),
])

# STRONG aug (middle)
try:
    from torchvision.transforms import TrivialAugmentWide
    strong_policy = [TrivialAugmentWide()]
except Exception:
    strong_policy = []  # fallback cleanly if not available

transform_train_strong = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    *strong_policy,
    transforms.ToTensor(),
    transforms.Normalize((0.4914,0.4822,0.4465),
                         (0.2023,0.1994,0.2010)),
    transforms.RandomErasing(p=0.25),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914,0.4822,0.4465),
                         (0.2023,0.1994,0.2010)),
])

trainset = torchvision.datasets.CIFAR10(root="./data", train=True,  download=True, transform=transform_train_light)
testset  = torchvision.datasets.CIFAR10(root="./data", train=False, download=True, transform=transform_test)

trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=cfg.batch_size, shuffle=True,
    num_workers=2, pin_memory=pin_mem, persistent_workers=True
)
testloader  = torch.utils.data.DataLoader(
    testset, batch_size=cfg.batch_size, shuffle=False,
    num_workers=2, pin_memory=pin_mem, persistent_workers=True
)

# =========================
# Polynomial softplus approx (degree-4) — use your 91% coefficients
# =========================
class PolyAct4(nn.Module):
    """
    y = s * (A x^4 + B x^3 + C x^2 + D x + E) + b, with x clamped to [-clip, clip].
    A..E fixed; s,b learnable (can be folded for inference).
    """
    def __init__(self, A, B, C, D, E, clip=6.0, init_scale=1.0, init_bias=0.0):
        super().__init__()
        self.register_buffer('A', torch.tensor(float(A)))
        self.register_buffer('B', torch.tensor(float(B)))
        self.register_buffer('C', torch.tensor(float(C)))
        self.register_buffer('D', torch.tensor(float(D)))
        self.register_buffer('E', torch.tensor(float(E)))
        self.clip = float(clip)
        self.scale = nn.Parameter(torch.tensor(float(init_scale)))
        self.bias  = nn.Parameter(torch.tensor(float(init_bias)))

    def forward(self, x):
        x = torch.clamp(x, -self.clip, self.clip)
        x2 = x * x; x3 = x2 * x; x4 = x2 * x2
        y = self.A * x4 + self.B * x3 + self.C * x2 + self.D * x + self.E
        return self.scale * y + self.bias

def make_poly():
    # per-layer instance (important)
    return PolyAct4(
        A=-0.00068481,
        B=-1.59833239e-17,
        C=0.0887234775,
        D=0.5,
        E=0.738099333,
        clip=cfg.act_clip,
        init_scale=1.0, init_bias=0.0
    )

# =========================
# PreAct-ResNet-18 (BN -> Poly -> Conv), CIFAR stem
# =========================
class PreActBlock(nn.Module):
    expansion = 1
    def __init__(self, in_planes, planes, stride=1, make_act=make_poly):
        super().__init__()
        self.bn1  = nn.BatchNorm2d(in_planes)
        self.act1 = make_act()
        self.conv1= nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)

        self.bn2  = nn.BatchNorm2d(planes)
        self.act2 = make_act()
        self.conv2= nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)

        self.downsample = None
        if stride != 1 or in_planes != planes*self.expansion:
            self.downsample = nn.Conv2d(in_planes, planes*self.expansion, kernel_size=1, stride=stride, bias=False)

    def forward(self, x):
        out = self.act1(self.bn1(x))
        sc_in = out
        out = self.conv1(out)
        out = self.conv2(self.act2(self.bn2(out)))
        shortcut = x if self.downsample is None else self.downsample(sc_in)
        return out + shortcut

class PreActResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10, make_act=make_poly):
        super().__init__()
        self.in_planes = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)

        self.layer1 = self._make_layer(block, 64,  layers[0], stride=1, make_act=make_act)
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2, make_act=make_act)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2, make_act=make_act)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2, make_act=make_act)

        self.bn_last  = nn.BatchNorm2d(512 * block.expansion)
        self.act_last = make_act()

        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.fc      = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.Linear):
                nn.init.kaiming_uniform_(m.weight, a=math.sqrt(5))

    def _make_layer(self, block, planes, blocks, stride, make_act):
        layers = [block(self.in_planes, planes, stride=stride, make_act=make_act)]
        self.in_planes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.in_planes, planes, stride=1, make_act=make_act))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.act_last(self.bn_last(x))
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

def PreActResNet18(num_classes=10):
    return PreActResNet(PreActBlock, [2,2,2,2], num_classes=num_classes, make_act=make_poly)

model = PreActResNet18(num_classes=10).to(device)

# =========================
# Optimizer: AdamW with WD hygiene (no WD on BN/affine or PolyAct scale/bias)
# =========================
def build_param_groups(m):
    decay, no_decay = [], []
    for n, p in m.named_parameters():
        if not p.requires_grad:
            continue
        if p.ndim == 1 or n.endswith(".bias") or "bn" in n.lower() or n.endswith(".scale"):
            no_decay.append(p)
        else:
            decay.append(p)
    return [
        {"params": decay, "weight_decay": cfg.weight_decay},
        {"params": no_decay, "weight_decay": 0.0},
    ]

optimizer = optim.AdamW(build_param_groups(model), lr=cfg.lr)

# Cosine LR over full training
scheduler = CosineAnnealingLR(optimizer, T_max=cfg.epochs, eta_min=1e-5)

# AMP (new API)
scaler = torch.amp.GradScaler('cuda', enabled=cfg.amp)

# EMA
class ModelEMA:
    def __init__(self, model, decay=0.999):
        self.ema = copy.deepcopy(model).to(device).eval()
        for p in self.ema.parameters():
            p.requires_grad_(False)
        self.decay = decay
    @torch.no_grad()
    def update(self, model):
        msd = model.state_dict()
        for k, v in self.ema.state_dict().items():
            if v.dtype.is_floating_point:
                v.copy_(v * self.decay + msd[k] * (1.0 - self.decay))
            else:
                v.copy_(msd[k])
    def state_dict(self):
        return self.ema.state_dict()

ema = ModelEMA(model, decay=cfg.ema_decay)

# =========================
# MixUp utilities
# =========================
def mixup_data(x, y, alpha):
    if alpha <= 0.0:
        return x, y, y, 1.0
    lam = torch.distributions.Beta(alpha, alpha).sample().item()
    bs  = x.size(0)
    idx = torch.randperm(bs, device=x.device)
    mixed_x = lam * x + (1 - lam) * x[idx]
    y_a, y_b = y, y[idx]
    return mixed_x, y_a, y_b, lam

def mixup_criterion(crit, pred, y_a, y_b, lam):
    return lam * crit(pred, y_a) + (1 - lam) * crit(pred, y_b)

criterion = nn.CrossEntropyLoss()  # no label smoothing when using MixUp

# =========================
# Eval helper (per-sample mean loss)
# =========================
@torch.no_grad()
def evaluate(eval_model):
    eval_model.eval()
    total, correct, loss_sum = 0, 0, 0.0
    for x, y in testloader:
        x = x.to(device, non_blocking=pin_mem)
        y = y.to(device, non_blocking=pin_mem)
        with torch.amp.autocast('cuda', enabled=cfg.amp):
            logits = eval_model(x)
            loss = criterion(logits, y)
        loss_sum += loss.item() * y.size(0)
        pred = logits.argmax(1)
        correct += (pred == y).sum().item()
        total += y.size(0)
    return loss_sum / total, 100.0 * correct / total

# BN calibration (refresh running stats on EMA before final eval)
@torch.no_grad()
def calibrate_bn(model, loader, iters=200):
    model.train()
    n = 0
    for x, _ in loader:
        x = x.to(device, non_blocking=pin_mem)
        model(x)
        n += 1
        if n >= iters:
            break
    model.eval()

# =========================
# Training
# =========================
phase1 = cfg.phase1_epochs
phase3 = max(1, int(cfg.phase3_ratio * cfg.epochs))
phase2 = max(1, cfg.epochs - phase1 - phase3)  # middle

best_acc = 0.0

for epoch in range(1, cfg.epochs + 1):
    # --- Augmentation schedule ---
    if epoch <= phase1:
        trainset.transform = transform_train_light
        mix_alpha = 0.0
        phase_name = "LIGHT"
    elif epoch <= phase1 + phase2:
        trainset.transform = transform_train_strong
        # MixUp ramp during the first part of phase 2
        ramp_ep = min(cfg.mixup_ramp_epochs, phase2)
        e_in_phase2 = epoch - phase1
        if e_in_phase2 <= ramp_ep:
            mix_alpha = cfg.mixup_target_alpha * (e_in_phase2 / ramp_ep)
        else:
            mix_alpha = cfg.mixup_target_alpha
        phase_name = "STRONG"
    else:
        trainset.transform = transform_train_light
        mix_alpha = 0.0
        phase_name = "LIGHT (final)"

    model.train()
    running_loss = 0.0

    loop = tqdm(trainloader, desc=f"Epoch {epoch}/{cfg.epochs} [{phase_name}]",
                leave=False)
    for x, y in loop:
        x = x.to(device, non_blocking=pin_mem)
        y = y.to(device, non_blocking=pin_mem)

        if mix_alpha > 0.0:
            x, y_a, y_b, lam = mixup_data(x, y, mix_alpha)
        else:
            y_a = y_b = y; lam = 1.0

        optimizer.zero_grad(set_to_none=True)
        with torch.amp.autocast('cuda', enabled=cfg.amp):
            logits = model(x)
            loss = mixup_criterion(criterion, logits, y_a, y_b, lam)

        scaler.scale(loss).backward()
        # (No grad clipping here; remove cap for higher ceiling)
        scaler.step(optimizer)
        scaler.update()

        ema.update(model)

        running_loss += loss.item() * x.size(0)
        loop.set_postfix(loss=float(loss),
                         lr=optimizer.param_groups[0]['lr'],
                         mixup_alpha=round(mix_alpha, 3))

    scheduler.step()
    train_loss = running_loss / len(trainset)

    # Evaluate raw model briefly at the very start, then EMA
    eval_model = model if epoch <= 3 else ema.ema
    val_loss, val_acc = evaluate(eval_model)

    print(f"Epoch [{epoch}/{cfg.epochs}] | Train Loss: {train_loss:.4f} | "
          f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}% | MixUp α: {mix_alpha:.3f}")

    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), cfg.save_std)
        torch.save(ema.state_dict(), cfg.save_ema)

# =========================
# Final eval: calibrate BN on EMA, then evaluate
# =========================
if os.path.exists(cfg.save_ema):
    ema_state = torch.load(cfg.save_ema, map_location=device)
    ema.ema.load_state_dict(ema_state)
    calibrate_bn(ema.ema, trainloader, iters=200)
    loss_e, acc_e = evaluate(ema.ema)
    print(f"\nEMA Model (BN-calibrated) -> Acc: {acc_e:.2f}% | Loss: {loss_e:.4f}")
else:
    print("\nNo EMA checkpoint found.")

if os.path.exists(cfg.save_std):
    model.load_state_dict(torch.load(cfg.save_std, map_location=device))
    loss_m, acc_m = evaluate(model)
    print(f"Last-saved Model          -> Acc: {acc_m:.2f}% | Loss: {loss_m:.4f}")
else:
    print("No standard model checkpoint found.")

print(f"\nSaved paths:\n - {cfg.save_std}\n - {cfg.save_ema}")



Files already downloaded and verified
Files already downloaded and verified


Epoch 1/350 [LIGHT]:   0%|          | 0/391 [00:20<?, ?it/s]

Epoch [1/350] | Train Loss: 1.6640 | Val Loss: 1.7936 | Val Acc: 36.33% | MixUp α: 0.000


Epoch 2/350 [LIGHT]:   0%|          | 0/391 [00:00<?, ?it/s]

Epoch [2/350] | Train Loss: 1.3105 | Val Loss: 1.2515 | Val Acc: 54.42% | MixUp α: 0.000


...

Epoch 192/350 [STRONG]:   0%|          | 0/391 [00:00<?, ?it/s]

Epoch [192/350] | Train Loss: 0.4004 | Val Loss: 0.2187 | Val Acc: 94.54% | MixUp α: 0.200


Epoch 193/350 [STRONG]:   0%|          | 0/391 [00:00<?, ?it/s]

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x000002DC0EEA69E0>>
Traceback (most recent call last):
  File "C:\Users\zelkh\research\lib\site-packages\ipykernel\ipkernel.py", line 781, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


KeyboardInterrupt: 

In [29]:
import os
import csv

def save_weights_to_csv(model, directory="my_model_weights"):
    # Create the directory if it does not exist
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    # Iterate over each parameter in the model
    for name, param in model.named_parameters():
        # Convert the parameter tensor to a flattened numpy array
        weights = param.data.cpu().numpy().flatten()
        
        # Replace dots in the parameter name with underscores and form the full file path
        filename = f"{directory}/{name.replace('.', '_')}.csv"
        
        # Open the CSV file and write the weights
        with open(filename, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(weights)  # Write weights as a single row
        
        print(f"Weights saved to {filename}")
    
    print("Weight extraction complete!")

# Call this function with your model variable
save_weights_to_csv(model)

Weights saved to my_model_weights/fc1_weight.csv
Weights saved to my_model_weights/fc1_bias.csv
Weights saved to my_model_weights/fc2_weight.csv
Weights saved to my_model_weights/fc2_bias.csv
Weight extraction complete!


In [36]:
import torch
import os
import csv

# Create directory for weights if it doesn't exist
os.makedirs('my_model_weights', exist_ok=True)

# Function to save weights to CSV
def save_to_csv(tensor, filename):
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        # Flatten the tensor and write as a single row
        writer.writerow(tensor.flatten().numpy())

# Load the trained model
activation_fn = PolyAct4()
model = CIFAR_CNN(activation_fn=activation_fn)
model.load_state_dict(torch.load('cifar10_trained_model.pth'))

# Method to get layer by name
def get_layer_by_name(model, name):
    modules = name.split('.')
    m = model
    for mod in modules:
        m = getattr(m, mod)
    return m

# Extract and save layer weights
block_layers = [
    'block1.0', 'block1.1', 'block1.3', 'block1.4',
    'block2.0', 'block2.1', 'block2.3', 'block2.4',
    'block3.0', 'block3.1', 'block3.3', 'block3.4',
    'block4.0', 'block4.1', 'block4.3', 'block4.4'
]

for layer_name in block_layers:
    try:
        layer = get_layer_by_name(model, layer_name)
        if isinstance(layer, (nn.Conv2d, nn.BatchNorm2d)):
            if hasattr(layer, 'weight'):
                save_to_csv(layer.weight.data, f'my_model_weights/{layer_name.replace(".", "_")}_weight.csv')
            if hasattr(layer, 'bias'):
                save_to_csv(layer.bias.data, f'my_model_weights/{layer_name.replace(".", "_")}_bias.csv')
            if isinstance(layer, nn.BatchNorm2d):
                save_to_csv(layer.running_mean, f'my_model_weights/{layer_name.replace(".", "_")}_running_mean.csv')
                save_to_csv(layer.running_var, f'my_model_weights/{layer_name.replace(".", "_")}_running_var.csv')
    except Exception as e:
        print(f"Error processing {layer_name}: {e}")

# Extract and save FC layer weights and BatchNorm
fc_layers = ['fc1', 'fc_bn', 'fc2']
for layer_name in fc_layers:
    try:
        layer = get_layer_by_name(model, layer_name)
        if isinstance(layer, nn.Linear):
            save_to_csv(layer.weight.data, f'my_model_weights/{layer_name}_weight.csv')
            save_to_csv(layer.bias.data, f'my_model_weights/{layer_name}_bias.csv')
        elif isinstance(layer, nn.BatchNorm1d):
            save_to_csv(layer.weight.data, f'my_model_weights/{layer_name}_weight.csv')
            save_to_csv(layer.bias.data, f'my_model_weights/{layer_name}_bias.csv')
            save_to_csv(layer.running_mean, f'my_model_weights/{layer_name}_running_mean.csv')
            save_to_csv(layer.running_var, f'my_model_weights/{layer_name}_running_var.csv')
    except Exception as e:
        print(f"Error processing {layer_name}: {e}")

print("Weight extraction complete!")

/tmp/ipykernel_4088899/672587445.py:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('cifar10_trained_model.pth'))


Weight extraction complete!


In [37]:
import os
import csv
import numpy as np

def preview_csv(filename, max_elements=10):
    with open(filename, 'r') as csvfile:
        reader = csv.reader(csvfile)
        data = list(reader)[0]  # First (and only) row
        data = [float(x) for x in data]
        
        print(f"File: {filename}")
        print(f"Total elements: {len(data)}")
        print("First few elements:", data[:max_elements])
        print("Shape:", len(data))
        print("Min:", min(data))
        print("Max:", max(data))
        print("Mean:", np.mean(data))
        print("Std Dev:", np.std(data))
        print("-" * 50)

# Directory containing the weights
weight_dir = 'my_model_weights'

# List all CSV files
csv_files = [f for f in os.listdir(weight_dir) if f.endswith('.csv')]

# Sort files for easier reading
csv_files.sort()

# Preview each CSV file
for filename in csv_files:
    full_path = os.path.join(weight_dir, filename)
    preview_csv(full_path)

File: my_model_weights/block1_0_bias.csv
Total elements: 48
First few elements: [-0.14158371, 0.086564094, 0.03638618, 0.15626998, 0.0087235095, -0.01983237, 0.077582106, -0.23901737, -0.066613145, 0.156942]
Shape: 48
Min: -0.47316253
Max: 0.2462121
Mean: -0.01763541479375
Std Dev: 0.14122533519858566
--------------------------------------------------
File: my_model_weights/block1_0_weight.csv
Total elements: 1296
First few elements: [-0.408994, -0.27043045, -0.1315946, 0.15104751, 0.6335488, 0.010418372, -0.24804702, 0.08897946, 0.2502869, 0.7613002]
Shape: 1296
Min: -1.820785
Max: 1.0271152
Mean: -0.00042038506412036864
Std Dev: 0.37214929194342206
--------------------------------------------------
File: my_model_weights/block1_1_bias.csv
Total elements: 48
First few elements: [2.239237, -1.6459881, -0.7095814, 2.4580042, -2.64024, -0.07478788, 2.497212, -1.5947889, -0.06567605, 0.6825575]
Shape: 48
Min: -2.7919664
Max: 3.4301686
Mean: -0.23573333333333335
Std Dev: 1.578670837339114


In [23]:
# Assuming `model` is your trained or loaded model
save_weights_to_csv(model, directory="my_model_weights")


Weights saved to my_model_weights/fc1_weight.csv
Weights saved to my_model_weights/fc1_bias.csv
Weights saved to my_model_weights/fc2_weight.csv
Weights saved to my_model_weights/fc2_bias.csv


In [24]:
import torch
import csv
import numpy as np

# Function to save a subset of CIFAR-10 data
def save_cifar10_data_subset(dataloader, percentage=10, filename="cifar10_subset.csv"):
    total_batches = len(dataloader)
    batches_to_save = int((percentage / 100) * total_batches)
    
    with open(filename, 'w', newline='') as file:
        writer = csv.writer(file)
        
        for i, (images, labels) in enumerate(dataloader):
            if i >= batches_to_save:
                break
            # Flatten images and convert them to numpy arrays
            images = images.numpy().reshape(images.shape[0], -1)
            labels = labels.numpy()
            
            # Write each image and label pair to the CSV file
            for img, label in zip(images, labels):
                # Optionally, append label to the image data
                img_with_label = np.append(img, label)
                writer.writerow(img_with_label)
    
    print(f"Saved {batches_to_save} batches of data to {filename}")

# Example usage
save_cifar10_data_subset(testloader, 10)


Saved 1000 batches of data to cifar10_subset.csv


In [3]:
# Install TenSEAL if not done
!pip install tenseal


In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# Dataset preparation
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False)

# Polynomial activation
class PolyAct4(nn.Module):
    def __init__(self, A, B, C, D, E):
        super().__init__()
        self.A = A
        self.B = B
        self.C = C
        self.D = D
        self.E = E

    def forward(self, x):
        x = torch.clamp(x, min=-6, max=6)
        return self.A * x**4 + self.B * x**3 + self.C * x**2 + self.D * x + self.E

poly4 = PolyAct4(
    A=-0.00068481,
    B=0.0,  # optional: skip B term to simplify
    C=0.0887234775,
    D=0.5,
    E=0.738099333
)

# Simple 2-layer model
class SimpleNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(3072, 512)
        self.act = poly4
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # flatten
        x = self.fc1(x)
        x = self.act(x)
        x = self.fc2(x)
        return x

model = SimpleNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training
for epoch in range(5):  # Just 5 epochs to test concept
    model.train()
    for images, labels in trainloader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}/5 done ✅")

# Save weights
torch.save(model.state_dict(), 'simple_model.pth')
print("Simple model saved! ✅")


Files already downloaded and verified
Files already downloaded and verified
Epoch 1/5 done ✅
Epoch 2/5 done ✅
Epoch 3/5 done ✅
Epoch 4/5 done ✅
Epoch 5/5 done ✅
Simple model saved! ✅
